In [54]:
from utils.dataset import set_b_dataset, set_b_dataclass
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.models as models
from random import sample
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader


PROJECT_DIR = '/Users/nattapolchanpaisit/Documents/GitHub/Algorithm/SC1015-PROJECT/'
N_TRAIN = 300
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
BATCH_SIZE = 32
EPOCH = 10
F_LOW = 200
F_HIGH = 400

train_id = tuple(sample([i for i in range(461)], N_TRAIN))
test_id = tuple(set([i for i in range(461)])-set(train_id))
writer = SummaryWriter(f'runs/{F_LOW}_{F_HIGH}_Hz')
trainset = set_b_dataclass(PROJECT_DIR, output_width=200, list_id=train_id)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
testset = set_b_dataclass(PROJECT_DIR, output_width=200, list_id=test_id)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)

model = torchvision.models.resnet18(False)
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = torch.nn.Linear(512, 3)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [62]:
def images_to_probs(model, image, idx):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    image = image[idx][0].view(1,1,output.size(1), output.size(2))
    output = model(image)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    Generates matplotlib Figure using a trained network, along with images
    and labels from a batch, that shows the network's top prediction along
    with its probability, alongside the actual label, coloring this
    information based on whether the prediction was correct or not.
    Uses the "images_to_probs" function.
    '''
    classes = ['normal', 'murmur', 'extrastole']
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        pred, prob = images_to_probs(net, images, idx)
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        ax.imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[pred],
            prob * 100.0,
            classes[labels[idx]]),
                    color=("green" if pred==labels[idx].item() else "red"))
    return fig

In [73]:
def train_cnn_stft():
    for epoch in range(EPOCH):
        running_loss = 0
        running_val_loss = 0
        running_val_acc = 0
        running_acc = 0
        total_murmur_acc = 0
        running_murmur_acc = 0
        for i, data in enumerate(trainloader):
            stft, labels = data
            stft = stft.to(torch.float32).to(DEVICE)
            stft = stft.view(-1, 1, stft.size(1), stft.size(2))
            label_one_hot = F.one_hot(labels, num_classes=3).to(torch.float32).to(DEVICE)
            model.train(True)
            grid = torchvision.utils.make_grid(stft)
            optimizer.zero_grad()
            outputs_one_hot = model(stft)
            outputs_one_hot = nn.Softmax()(outputs_one_hot)
            loss = torch.nn.CrossEntropyLoss()(label_one_hot, outputs_one_hot)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            model.train(False)
            output_labels = torch.argmax(outputs_one_hot, 1)
            running_acc += torch.sum(output_labels==labels)
            
        for i, data in enumerate(testloader):
            writer.add_image('image', grid, 0)
            writer.add_graph(model, stft)
            writer.close()
            stft, labels = data
            stft = stft.to(torch.float32).to(DEVICE)
            stft = stft.view(-1, 1, stft.size(1), stft.size(2))
            label_one_hot = F.one_hot(labels, num_classes=3).to(torch.float32).to(DEVICE)
            outputs_one_hot = model(stft)
            outputs_one_hot = nn.Softmax()(outputs_one_hot)
            loss = torch.nn.CrossEntropyLoss()(label_one_hot, outputs_one_hot)
            running_val_loss += loss.item()
            output_labels = torch.argmax(outputs_one_hot, 1)
            running_val_acc += torch.sum(output_labels==labels)
            filtered_murmur = labels[labels == 1]
            running_murmur_acc += torch.sum(output_labels*filtered_murmur==labels)
            total_murmur_acc += torch.sum(labels==torch.ones(size=(labels.size())) * 1)
            
        print(f'{epoch} : Train Loss', running_loss/trainset.__len__())
        print(f' : Train Accuracy', running_acc/trainset.__len__())
        print(f' : Validation Loss', running_val_loss/testset.__len__())
        print(f' : Validation Accuracy', running_val_acc/testset.__len__())
        print(f' : Validation Accuracy (Murmur)', running_murmur_acc/total_murmur_acc)
        writer.add_scalar('Loss/train',
                            running_loss / trainset.__len__(),
                            epoch * len(trainloader))
        writer.add_scalar('Loss/val',
                            running_val_loss / testset.__len__(),
                            epoch * len(trainloader))
        writer.add_scalar('Accuracy/train',
                            running_loss / trainset.__len__(),
                            epoch * len(trainloader))
        writer.add_scalar('Accuracy/val',
                            running_val_loss / testset.__len__(),
                            epoch * len(trainloader))
        
        # writer.add_figure('predictions vs. actuals',
        #                     plot_classes_preds(model, testset, labels),
        #                     global_step=epoch * len(trainloader) + i)

In [74]:
train_cnn_stft()

/var/folders/mk/w2mwvrjn5sn9g2fty5rrgzd40000gn/T/ipykernel_66434/3278987254.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs_one_hot = nn.Softmax()(outputs_one_hot)
/var/folders/mk/w2mwvrjn5sn9g2fty5rrgzd40000gn/T/ipykernel_66434/3278987254.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs_one_hot = nn.Softmax()(outputs_one_hot)


RuntimeError: The size of tensor a (32) must match the size of tensor b (7) at non-singleton dimension 0